**What does this notebook do?**
- Load the exported CGM values from NutriSense
- Print what days are included in the dataset
- Calculate simple statistics for every day in the dataset
- Calculate the average glucose value across all days
- Plot all the data, highlighting the average across all days

In [ ]:
import pandas as pd
import plotly.express as px
import datetime

# Read in CSV file
df = pd.read_csv('export.csv')

# Remove "time zone offset" from "occurred_at" column and add new "occurred_at_day" column
df['occurred_at_day'] = df['occurred_at'].apply(lambda x: x[:len(x) - 15])
df['occurred_at'] = df['occurred_at'].apply(lambda x: x[:len(x) - 6])
df.head()

In [ ]:
# Print all days with data
daysWithData = df['occurred_at_day'].unique()
print(daysWithData)

In [ ]:
# Create a new dataset that only contains glucose measurements
gm = df[df['class']=='GlucoseMeasurement']
gm.head()

In [ ]:
# Calculate a few simple metrics for every day

# Create a data frame that will hold the metrics
summaryData = pd.DataFrame(columns=["day","averageGlucose", "medianGlucose", "minGlucose", "maxGlucose"])

# Loop through all days in the dataset and calulate metrics
for day in daysWithData:
    gmDataForOneDay = gm[gm['occurred_at_day']==day]
    averageGlucose = int(round(gmDataForOneDay['value'].mean()))
    medianGlucose = int(round(gmDataForOneDay['value'].median()))
    maxGlucose = int(round(gmDataForOneDay['value'].max()))
    minGlucose = int(round(gmDataForOneDay['value'].min()))
    # Add to data frame
    summaryData = summaryData.append({'day':day, 'averageGlucose':averageGlucose, 'medianGlucose':medianGlucose, "minGlucose":minGlucose, "maxGlucose":maxGlucose}, ignore_index=True)

summaryData.head(n=10)

In [ ]:
# Calculate average glucose across all days
averageGlucoseAcrossAllDays = int(round(gm['value'].mean()))
print("Average Glucose for " + str(len(daysWithData)) + " days: " + str(averageGlucoseAcrossAllDays))

In [ ]:
# Chart the data for all days
fig = px.line(gm, x = "occurred_at", y="value")

# Draw the average as well
fig.add_shape(type="line", xref="x", yref="y", x0=gm.occurred_at.min(), y0=averageGlucoseAcrossAllDays, x1=gm.occurred_at.max() , y1=averageGlucoseAcrossAllDays, line_color="black",)

fig.show()